## 5.1 ドメインモデルの見直し

2.6 で作成したドメインモデル（疑似コード）は以下。これをコードにしていく:

```txt
context: Order-Taking

// ------------------
// 単純型
// ------------------

// 製品コード
data ProductCode = Widget OR GizmoCode
data WiedgetCode = string starting with "W" than 4 digits
data GizmoCode = ...

// 注文数量
data OrderQuantity = UnitQuantity OR KilogramQuantity
data UnitQuantity = ...
data KilogramQuantity = ...

// ------------------
// 注文のライフサイクル
// ------------------

// 未検証の状態
data ValidatedOrder = ...
data ValidatedOrderLine = ...

// 価格計算済みの状態
data PricedOrder = ...
data PricedOrderLine = ...

// 出力イベント
data OrderAcknowledgementSent = ...
data OrderPlaced = ...
data BillableOrderPlaced = ...

// ------------------
// ワークフロー
// ------------------

workflow "Place Order" =
    input: UnvalidatedOrder
    output (on success):
        OrderAcknowledgmentSent
        AND OrderPlaced (to send to shipping)
        AND BillableOrderPlaced (to send to billing)
    output (on error):
        InvalidOrder

// ...
```

# 5.2 ドメインモデルのパターンを見る

* 単純な値 (単純型を使用)
* AND による値の組み合わせ (レコードを使用)
* OR による選択肢 (選択型を使用)
* ワークフロー (関数を使用)

## 5.3.1 単一ケース共同体の利用

In [1]:
type CustomerId = CustomerId of int
type OrderId = OrderId of int

// struct
let customerId = CustomerId 42
let orderId = OrderId 42

// destruct
let (CustomerId innerValue) = customerId
printfn "innerValue is %i" innerValue

The below script needs to be able to find the current output cell; this is an easy method to get it.

innerValue is 42


### 5.7.3 エンティティに対する等値性の実装

連続的な同一性を定義するには以下のように F# のオブジェクト指向の構文を使う

In [2]:
type Undefined = exn // exn は例外型

type ContactId = ContactId of int
type PhoneNumber = PhoneNumber of string
type EmailAddress = EmailAddress of string

// 1. Equals メソッドをオーバーライド
// 2. GetHashCode メソッドをオーバーライド
// 3. CustomEquality と NoComparison 属性を型に追加して、デフォルトの動作を変更したいことをコンパイラに伝える
[<CustomEquality; NoComparison>]
type Contact = {
    ContactId : ContactId
    PhoneNumber : PhoneNumber
    EmailAddress : EmailAddress
    }
    with
    override this.Equals(obj) =
        match obj with
        | :? Contact as c -> this.ContactId = c.ContactId
        | _ -> false
    override this.GetHashCode() =
        hash this.ContactId

// usage
let contactId = ContactId 1

let contact1 = {
    ContactId = contactId
    PhoneNumber = PhoneNumber "123-456-7890"
    EmailAddress = EmailAddress "bob@example.com"
}

let contact2 = {
    ContactId = contactId
    PhoneNumber = PhoneNumber "123-456-7890"
    EmailAddress = EmailAddress "robert@example.com"
}

printfn "%b" (contact1 = contact2)

[<NoEquality; NoComparison>]
type NoEqualityContact = {
    ContactId : ContactId
    PhoneNumber : PhoneNumber
    EmailAddress : EmailAddress
}

// usage
let contactNoEq1: NoEqualityContact  = {
    ContactId = contactId
    PhoneNumber = PhoneNumber "123-456-7890"
    EmailAddress = EmailAddress "bob@example.com"
}

let contactNoEq2: NoEqualityContact = {
    ContactId = contactId
    PhoneNumber = PhoneNumber "123-456-7890"
    EmailAddress = EmailAddress "robert@example.com"
}

// Compile Error!
// printfn "%b" (contactNoEq1 = contactNoEq2)

// No complie error
printfn "%b" (contactNoEq1.ContactId = contactNoEq2.ContactId)

// Composite Key comparison
type OrderId = OrderId of string
type ProductId = ProductId of string

[<NoEquality; NoComparison>]
type OrderLine = {
    OrderId : OrderId
    ProductId : ProductId
    Qty : int
    }
    with
    member this.Key =
        (this.OrderId, this.ProductId)

// usage
let line1 = {
    OrderId = OrderId "xxx"
    ProductId = ProductId "yyy"
    Qty = 0
}

let line2 = {
    OrderId = OrderId "xxx"
    ProductId = ProductId "zzz"
    Qty = 0
}

printfn "%b" (line1.Key = line2.Key)

true
true
false


## 5.9 すべてを組み合わせる

In [3]:
// workflow "Place Order" =
//     input: UnvalidatedOrder
//     output (on success):
//         OrderAcknowledgmentSent
//         AND OrderPlaced (to send to shipping)
//         AND BillableOrderPlaced (to send to billing)
//     output (on error):
//         InvalidOrder

type UnvalidatedOrder = {
    OrderId : string
    CustomerInfo : string
    ShippingAddres : string
    // ...
}

type PlaceOrderEvents = {
    AcknowledgementSent : Undefined
    OrderPlaced : Undefined
    BillableOrderPlaced : Undefined
}

type PlaceOrderError =
    | ValidationError of ValidationError list
    // ...
and ValidationError = {
    FieldName : string
    ErrorDescription : string
}

type PlaceOrder =
    UnvalidatedOrder -> Result<PlaceOrderEvents, PlaceOrderError>